In [1]:
import pandas as pd
import os
from datetime import datetime, timedelta

config_file_path = 'Config.csv'
temp_file_path = 'Temp.csv'

def set_flags(x,limit_type,r_limit,y_limit,debug=False):
    flag = 'G'
    if limit_type == 'UCL':
        if x >= r_limit:
            flag = 'R'
        elif x>= y_limit:
            flag = 'Y'
    else:
        if x <= r_limit:
            flag = 'R'
        elif x<= y_limit:
            flag = 'Y'       

    if debug: print(f'        Type={limit_type}, Value={x}, R={r_limit}, Y={y_limit}, Flag={flag}') 
    return flag

def load_prod_data(server,product,details,limits,debug=False):

    if debug: print(limits)
    tech = details['TECH']
    prod = details['PART']
    rev = details['REV']
    idv_unit = details['IDV_UNIT']
    sicc_unit = details['SICC_UNIT']
    cap_unit = details['CAPABILITY_UNIT']
    cdyn_unit = details['CDYN_UNIT']
    
    if debug: print(f'Product={product}')
    
    folders = r'\Actuals\Last_49_Days'
    if tech=='P1273':
        folders = r'\Actuals\Last_49Days'
    
    file_path = os.path.join(r'\\'+server,tech+r'_Data'+folders,prod+'.csv')
    
    # debug = True
    df = pd.read_csv(file_path)
    # if debug: print(df.head())

    # if debug:
    #     col_check = 1
    #     num_rows = df.shape[0]
    #     print(f"Column check {col_check} = {num_rows}")
    #     col_check +=1
        
    df = df[df['PROCESS_REV'] == rev]

    # if debug:
    #     num_rows = df.shape[0]
    #     print(f"Column check {col_check} = {num_rows}")
    #     col_check +=1

    df['SORT_DATE'] = pd.to_datetime(df['SORT_DATE'], errors='coerce')
    # Calculate the start and end dates for the last 4 full weeks
    today = datetime.today()
    start_of_this_week = today - timedelta(days=today.weekday() + 1)
    start_of_4th_last_full_week = start_of_this_week - timedelta(weeks=4)

    # Filter the DataFrame for the last 4 full weeks
    df = df[
        (df['SORT_DATE'] >= start_of_4th_last_full_week)
    ]
    # if debug:
    #     num_rows = df.shape[0]
    #     print(f"Column check {col_check} = {num_rows}")
    #     col_check +=1

    possible_columns = ['IDV', 'SICC', 'CAPABILITY', 'CDYN']
    columns_of_interest = []
    
    for col in possible_columns:        
        if pd.notna(details[col]) and details[col] != '':
            columns_of_interest.append(col)
            df[col] = df[details[col]]
    
    means = df.groupby('FAB')[columns_of_interest].mean().reset_index()
    means['TECH'] = tech
    means['PRODUCT'] = product
    # print(means)

    # for col in columns_of_interest:
    #     if col == 'SICC':
    #         mult = 1000
    #         if sicc_unit != 'mA': mult=1
    #         means['SICC'] = means['SICC']*mult
    #         digits = 2
    #         # means['SICC'] = means['SICC'].round(2)
    #     elif col == 'CDYN':
    #         digits = 2
    #     else:
    #         if idv_unit == 'Mhz':
    #             digits = 0
    #         else:
    #             digits = 2
    #     means[col] = means[col].round(digits)
            

 

        
    
    # for col in columns_of_interest:
        # means[col+'_Flag'] = 'Green'
        
    for col in columns_of_interest:
    # print(col)
        if debug: print(f'  Parameter={col}')
        limit_type = limits[col+'_TYPE']
        target = details[col+'_TGT']
        r_limit = target*(1+limits[col+'_RED'])
        y_limit = target*(1+limits[col+'_YELLOW'])
        
        means[col+'_FLAG'] = means[col].apply(set_flags, args=(limit_type, r_limit, y_limit, debug))
    

    
    return means
    
    
def load_excel_to_dict(file_path,key_col):
    df = pd.read_csv(file_path)
    temp_dict = {}
    for _, row in df.iterrows():
        key = row[key_col]
        temp_dict[key] = row.drop(key_col).to_dict()
    return temp_dict
prod_dict = load_excel_to_dict('Config.csv','PRODUCT')
limit_dict = load_excel_to_dict('Tech_Limits.csv','TECH')
# prod_dict
server = 'rasinkul-desk'

temp_dfs = []
for product, details in prod_dict.items():
# if True:
#     product = 'RPL68'
    # details = prod_dict['RPL68']
    print(product)
    limits = limit_dict[details['TECH']]
    # product = details['PART']

    temp_df = load_prod_data(server=server, product=product, details=details,limits=limits, debug=False)

    temp_dfs.append(temp_df)
    
final_df = pd.concat(temp_dfs, ignore_index=True)

desired_order = ['TECH','PRODUCT','FAB','IDV','SICC','CAPABILITY','CDYN','IDV_FLAG','SICC_FLAG','CAPABILITY_FLAG','CDYN_FLAG']
    
final_df = final_df[desired_order]
final_df
final_df.to_csv('output.csv')

In [1]:
import Opeda_Scraper as opeda1

temp_dir = r"D:\Python\PDE_Weekly\Temp"

binsplit = opeda1.Opeda_Scraper(url=r'https://opeda.intel.com/binsplit',download_dir=temp_dir)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\Python\\PDE_Weekly\\Temp'

In [23]:

unique_opeda_prods = set()
for product, details in prod_dict.items():
    if pd.notna(details['OPEDA_PROD']):
        unique_opeda_prods.add(details['OPEDA_PROD'])

print(unique_opeda_prods)

{'PS2CV', '8PG3CV', 'PQFDCV', 'PQGCV'}


In [25]:
for opeda_prod in unique_opeda_prods:
    print(opeda_prod)
    binsplit.pull_product_data(opeda_prod)

PS2CV


ElementClickInterceptedException: Message: element click intercepted: Element <button _ngcontent-xdv-c309="" title="Download to Excel" class="btn btn-default px-1 py-0 border-0">...</button> is not clickable at point (2102, 222). Other element would receive the click: <div class="overlay-spinner" style="width: 1826.86px; height: 65.1429px; top: 210.902px; left: 304px; position: fixed; z-index: 10; background-color: rgba(255, 255, 255, 0.98); display: flex; justify-content: center; align-items: center;">...</div>
  (Session info: chrome=132.0.6834.160)
Stacktrace:
	GetHandleVerifier [0x011B74A3+25091]
	(No symbol) [0x0113DC04]
	(No symbol) [0x0101B373]
	(No symbol) [0x010654B8]
	(No symbol) [0x010638C4]
	(No symbol) [0x01061467]
	(No symbol) [0x0106076A]
	(No symbol) [0x010553A5]
	(No symbol) [0x01081F0C]
	(No symbol) [0x01054E44]
	(No symbol) [0x010821A4]
	(No symbol) [0x0109B49E]
	(No symbol) [0x01081CA6]
	(No symbol) [0x010531D5]
	(No symbol) [0x0105435D]
	GetHandleVerifier [0x014B07

In [162]:
import numpy as np
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment

In [163]:
wb = Workbook()
ws = wb.active

In [164]:
bold_font = Font(bold=True)
blue_font = Font(color="0000FF")
green_font = Font(color="00B050")
orange_font = Font(color="FF9900")
center_alignment = Alignment(horizontal="center", vertical="center")

In [165]:
ws['A1'] = 'Benchmark/Goal'
ws['A1'].font = bold_font
ws['A1'].alignment = center_alignment

ws['B1'] = 'F32'
ws['B1'].font = bold_font
ws['B1'].alignment = center_alignment

ws['C1'] = 'F28'
ws['C1'].font = bold_font
ws['C1'].alignment = center_alignment


In [166]:
prod_dict['RPL816']['IDV_UNIT']

'Mhz'

In [167]:
def get_value(df, prod_dict, product, fab, column):
    unit = prod_dict[product][column+'_UNIT']
    
    mult = 1
    if column == 'SICC':
        mult = 1000
        if unit != 'mA': mult=1
        # means['SICC'] = means['SICC']*mult
        digits = 2
    elif column == 'CDYN':
        digits = 2
    else:
        if unit == 'Mhz':
            digits = 0
        else:
            digits = 2
    if unit == "NONE": unit = ""
    print(unit)
    print(mult)
    print(digits)
    
    values = df[(df['PRODUCT'] == product) & (df['FAB'] == fab)][column].values
    value = values[0]*mult
    rnd_value = np.round(value, decimals=digits)
    formatted_value = f"{rnd_value:.{digits}f}{unit}"
    return formatted_value

def get_flag(df, product, fab, column):
    flag = df[(df['PRODUCT'] == product) & (df['FAB'] == fab)][column].values
    return flag[0] if len(flag) > 0 else None

In [170]:
get_value(final_df, prod_dict, 'GNRHCC', 'F34','SICC')

A
1
2


'1.08A'

In [ ]:
rows = [
    ['IDV > Target-0.5%',
     f""]
]

In [21]:
import win32com.client
import pythoncom
pythoncom.CoInitialize()  # Initialize COM for the current thread
excel = win32com.client.Dispatch("Excel.Application")
filePath = r"\\azshfs.intel.com\AZAnalysis$\1272_MAODATA\Config\PDE\dagarcia\PDE_Weekly\Customer_Ops_Scorecard.xlsm"
workbook = excel.Workbooks.Open(filePath)
excel.Application.Run("Convert")
excel.Quit()
pythoncom.CoUninitialize()